# 05 - 판다스 데이터프레임 집계

## 4.데이터프레임 집계

- 상세 데이터가 아닌 집계된 데이터에 대한 분석을 자주 요구하니 익숙해져야 할 내용입니다.
- sum(), mean(), max(), min(), count() 메소드를 사용해 지정한 열 또는 열들을 기준으로 집계합니다.
- 평균을 구하는 메소드가 avg()가 아닌 mean() 임을 주의하기 바랍니다.

In [1]:
# 라이브러리 불러오기
import pandas as pd

* Attrition 데이터 불러오기

**[Attrition 데이터 셋 정보]**

|	구분	|	변수 명	|	내용	|	type	|	비고	|
|	----	|	----	|	----	|	----	|	----	|
|	**Target**	|	**Attrition**	|	이직여부, Yes , No	|	범주	| 1- 이직, 0- 잔류		|
|	feature	|	Age	|	나이	|	숫자	|		|
|	feature	|	DistanceFromHome	|	집-직장 거리	|	숫자	|	마일	|
|	feature	|	EmployNumber	|	사번	|	숫자	| 	|
|	feature	|	Gender	|	성별	|	범주	| Male, Female		|
|	feature	|	JobSatisfaction	|	직무 만족도	|	범주	|	1 Low, 2 Medium, 3 High, 4 Very High	|
|	feature	|	MaritalStatus	|	결혼상태	|	범주	| Single, Married, Divorced		|
|	feature	|	MonthlyIncome	|	월급	|	숫자	| 달러	|
|	feature	|	OverTime	|	야근여부	|	범주	|	Yes, No	|
|	feature	|	PercentSalaryHike	|	전년대비 급여인상율	|	숫자	|	%	|
|	feature	|	TotalWorkingYears	|	총 경력 연수	|	숫자	|		|

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition_simple2.CSV'
data = pd.read_csv(path)  

# 상위 5개 확인
data.head(5)

,Attrition,Age,DistanceFromHome,EmployeeNumber,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


### 4.1 열 하나 집계

- 우선 특정 열의 값 합은 다음과 같이 구할 수 있습니다.

In [3]:
# MonthlyIncome 합계
data['MonthlyIncome'].sum()

7798045

In [4]:
# MonthlyIncome, TotalWorkingYears 각각의 평균
data[['MonthlyIncome', 'TotalWorkingYears']].mean()

MonthlyIncome        6520.104515
TotalWorkingYears      11.330268
dtype: float64

**1) 집계하기**

- 만일 day 별로 합을 구하고자 한다면 다음과 같이 합니다.
- 아래 결과 값 네 개를 더하면 전체 합이 됩니다.
- **as_index=True**를 설정(기본값)하면 집계 기준이 되는 열이 인덱스 열이 됩니다.
- 집계 결과가 data 열만 가지니 **시리즈**가 됩니다.

In [5]:
# MaritalStatus 별 Age 평균 --> 시리즈
data.groupby('MaritalStatus', as_index=True)['Age'].mean()

MaritalStatus
Divorced    37.522727
Married     37.704380
Single      35.460938
Name: Age, dtype: float64

- **[['data']].sum()**과 같이 하면 열이 여럿이라는 의미여서 결과가 **데이터프레임**이 됩니다.

In [6]:
# MaritalStatus 별 Age 평균 --> 데이터프레임
data.groupby('MaritalStatus', as_index=True)[['Age']].mean()

,Age
MaritalStatus,
Divorced,37.522727
Married,37.704380
Single,35.460938


- **as_index=False**를 설정하면 행 번호를 기반으로 한 정수 값이 인덱스로 설정됩니다.

In [7]:
# MaritalStatus 별 Age 평균 --> 데이터프레임
data.groupby('MaritalStatus', as_index=False)[['Age']].mean()

,MaritalStatus,Age
0,Divorced,37.522727
1,Married,37.704380
2,Single,35.460938


**2) 데이터프레임으로 선언**

- 집계 결과를 새로운 데이터프레임으로 선언하여 사용하는 경우가 많습니다.
- 집계된 결과를 반복해서 사용하거나, 분석 대상이 되는 경우 데이터프레임으로 선언함이 유익합니다.

In [8]:
data_mean = data.groupby('MaritalStatus', as_index=False)[['Age']].mean()

# 확인
data_mean

,MaritalStatus,Age
0,Divorced,37.522727
1,Married,37.704380
2,Single,35.460938


### 4.2 여러 열 집계

- 여러 열에 대한 집계를 같이 할 수 있습니다.
- **[ ['feature1', 'feature2'] ].sum()** 형태와 같이 집계 대상 열을 리스트로 지정합니다.

In [9]:
data_mean = data.groupby('MaritalStatus', as_index=False)[['Age','MonthlyIncome']].mean()

# 확인
data_mean

,MaritalStatus,Age,MonthlyIncome
0,Divorced,37.522727,6707.018939
1,Married,37.704380,6880.144161
2,Single,35.460938,5877.794271


- sum() 메소드 앞에 아무 열도 지정하지 않으면 **기준열 이외의 모든 열에 대한 집계**가 수행됩니다.

In [10]:
data_sum = data.groupby('MaritalStatus', as_index=False).sum()

# 확인
data_sum

,MaritalStatus,Attrition,Age,DistanceFromHome,EmployeeNumber,JobSatisfaction,MonthlyIncome,PercentSalaryHike,TotalWorkingYears
0,Divorced,23,9906,2404,266305,716,1770653,3958,3106
1,Married,69,20662,5295,584446,1468,3770319,8431,6470
2,Single,103,13617,3374,387862,1065,2257073,5852,3975


- **by=['feature1', 'feature2']** 과 같이 집계 기준 열을 여럿 설정할 수도 있습니다.

In [11]:
# 'MaritalStatus', 'Gender'별 나머지 열들 평균 조회
data_sum = data.groupby(['MaritalStatus', 'Gender'], as_index=False)[['Age','MonthlyIncome']].mean()

# 확인
data_sum

,MaritalStatus,Gender,Age,MonthlyIncome
0,Divorced,Female,37.010526,6626.315789
1,Divorced,Male,37.810651,6752.384615
2,Married,Female,38.774194,7301.493088
3,Married,Male,37.003021,6603.912387
4,Single,Female,35.261146,5963.445860
5,Single,Male,35.599119,5818.555066


### 4.3 여러 함수로 한꺼번에 집계

* df.groupby(  )**.agg(['함수1','함수2', ...])**

In [15]:
data_agg = data.groupby('MaritalStatus', as_index=False)[['MonthlyIncome', 'Age']].agg(['min','max','mean'])
# 확인
data_agg

MonthlyIncome                     Age               
                        min    max         mean min max       mean
MaritalStatus                                                     
Divorced               1129  19973  6707.018939  22  60  37.522727
Married                1052  19999  6880.144161  22  60  37.704380
Single                 1009  19926  5877.794271  18  59  35.460938

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] 데이터를 불러와 데이터프레임으로 저장합니다.

- 파일 경로 : https://raw.githubusercontent.com/DA4BAM/dataset/master/airquality_simple.csv

**[airquality_simple 데이터 셋 정보]**

- Ozone : 오존농도
- Solar.R: 태양복사열
- Wind: 풍속
- Temp: 기온
- Month: 월
- Day: 일

In [16]:
# 데이터 읽어오기
air = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/airquality_simple.csv")

# 상위 5개 확인
air.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41,190.0,7.4,67,5,1
1,36,118.0,8.0,72,5,2
2,12,149.0,12.6,74,5,3
3,18,313.0,11.5,62,5,4
4,19,NaN,14.3,56,5,5


[문2] 월별 Ozone, Wind, Temp 평균을 구해 봅시다.

In [22]:
air.groupby('Month', as_index=False)[['Ozone', 'Wind', 'Temp']].mean()

,Month,Ozone,Wind,Temp
0,5,22.677419,11.622581,65.548387
1,6,40.133333,10.266667,79.100000
2,7,56.870968,8.941935,83.903226
3,8,59.000000,8.793548,83.967742
4,9,31.166667,10.180000,76.900000


[문3] 월별 Ozone, Wind, Temp에 대해 최대, 최소, 평균, 표준편차 값을 구해 봅시다.

In [23]:
air.groupby('Month')[['Ozone', 'Wind', 'Temp']].agg(['min','max','mean','std'])

Ozone                            Wind                            Temp  \
        min  max       mean        std  min   max       mean       std  min   
Month                                                                         
5         1  115  22.677419  20.416312  5.7  20.1  11.622581  3.531450   56   
6        12   71  40.133333  18.339205  1.7  20.7  10.266667  3.769234   65   
7         7  135  56.870968  29.393584  4.1  14.9   8.941935  3.035981   73   
8         9  168  59.000000  37.704111  2.3  15.5   8.793548  3.225930   72   
9         7   96  31.166667  23.772025  2.8  16.6  10.180000  3.461254   63   

                                
      max       mean       std  
Month                           
5      81  65.548387  6.854870  
6      93  79.100000  6.598589  
7      92  83.903226  4.315513  
8      97  83.967742  6.585256  
9      93  76.900000  8.355671

### 4.3 종합실습

1) pandas 라이브러리를 pd 별칭을 주어 불러오세요.

In [24]:
# 라이브러리 불러오기
import pandas as pd

2) read_csv() 함수를 사용해 다음 경로의 파일을 불러와 **titanic** 데이터프레임을 만드세요.

- 파일 경로: 'https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic_simple.csv'

**[titanic_simple 데이터 셋 정보]**

- PassengerId : 승객번호
- Survived : 생존여부(1:생존, 0:사망)
- Pclass : 객실등급(1:1등급, 2:2등급, 3:3등급)
- Name : 승객이름
- Sex : 성별(male, female)
- Age : 나이
- Fare : 운임($)
- Embarked : 승선지역(Southhampton, Cherbourg, Queenstown)

In [25]:
# 파일 읽어오기
titanic = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic_simple.csv')

3) Fare의 중앙값을 확인해 봅시다.

In [27]:
titanic['Fare'].median()

14.4542

4) 승선지역(Embarked)별 평균 운임(Fare)을 구해 봅시다.

In [40]:
titanic.groupby('Embarked', as_index=False)['Fare'].mean()

,Embarked,Fare
0,Cherbourg,59.954144
1,Queenstown,13.276030
2,Southhampton,27.079812


5) 승선지역(Embarked)별, 성별(Sex)별 평균 운임(Fare)과 평균 나이(Age)를 tmp에 저장하고 조회하시오.

In [44]:
tmp = titanic.groupby(['Embarked', 'Sex'], as_index=False)[['Fare', 'Age']].mean()
tmp

,Embarked,Sex,Fare,Age
0,Cherbourg,female,75.169805,28.344262
1,Cherbourg,male,48.262109,32.998841
2,Queenstown,female,12.634958,24.291667
3,Queenstown,male,13.838922,30.937500
4,Southhampton,female,38.740929,27.771505
5,Southhampton,male,21.711996,30.291440


6) 5번의 결과로 부터 파악할 수 있는 내용은 무엇인가요?

In [34]:
#승선지역별 운임과 승객들의 나이는 정비례 관계에 있다

7) 객실등급(Pclass)별, 생존여부(Survived)별 나이(Age), 운임(Fare)의 최대, 최소, 평균, 표준편차를 tmp에 저장하고 조회하시오.

In [46]:
tmp = titanic.groupby(['Pclass','Survived'])[['Age','Fare']].agg(['min','max','mean','std'])
tmp

Age                                 Fare            \
                   min   max       mean        std      min       max   
Pclass Survived                                                         
1      0          2.00  71.0  43.695312  15.284243   0.0000  263.0000   
       1          0.92  80.0  35.368197  13.760017  25.9292  512.3292   
2      0         16.00  70.0  33.544444  12.151581   0.0000   73.5000   
       1          0.67  62.0  25.901566  14.837787  10.5000   65.0000   
3      0          1.00  74.0  26.555556  12.334882   0.0000   69.5500   
       1          0.42  63.0  20.646118  11.995047   0.0000   56.4958   

                                       
                      mean        std  
Pclass Survived                        
1      0         64.684007  60.662089  
       1         95.608029  85.286820  
2      0         19.412328  15.307175  
       1         22.055700  10.853502  
3      0         13.669364  12.118338  
       1         13.694887  10.692993